In [23]:
# PWM-MMIO-test in Programmable Logic (PL)

In [24]:
!pwd

/root/jupyter_notebooks/pynq-pwm


In [25]:
!ls

design.xdc  dpu.bit  dpu.hwh  dpu.xclbin  PWM-MMIO-test.ipynb


In [26]:
from pynq import Overlay
from pynq import MMIO
from pynq.lib import AxiGPIO

ol = Overlay("/root/jupyter_notebooks/pynq-pwm/dpu.bit")


In [42]:
gpio_1_ip = ol.ip_dict['axi_gpio_1']
gpio_out = AxiGPIO(gpio_1_ip).channel1

mask = 0xffffffff
gpio_out.write(0x1,mask)

In [43]:
base_address_0 = 0xA0010000     # axi_timer_0
base_address_1 = 0xA0030000     # axi_timer_1
base_address_2 = 0xA0040000     # axi_timer_2
base_address_3 = 0xA0050000     # axi_timer_3
address_range = 0x10000         # axi_timer_*

mmio_0 = MMIO(base_address_0, address_range)
mmio_1 = MMIO(base_address_1, address_range)
mmio_2 = MMIO(base_address_2, address_range)
mmio_3 = MMIO(base_address_3, address_range)


In [44]:

register_offset_TSCR0 = 0x0     # Timer 0 Control and Status Register
register_offset_TSCR1 = 0x10    # Timer 1 Control and Status Register
register_offset_TLR0 = 0x4      # Timer 0 Load Register
register_offset_TLR1 = 0x14     # Timer 1 Load Register

value_to_write = 0b1010010110   #PWM, TCSR, UDT, Autoreload, timer

mmio_0.write(register_offset_TSCR0, value_to_write)
mmio_0.write(register_offset_TSCR1, value_to_write)


In [45]:
_period_ = 20000  # 50Hz, 20ms
_pulse_ = 90  # 0-99
period = int((_period_ & 0x0ffff) * 100)
pulse = int((_pulse_ & 0x07f) * period / 100)

mmio_0.write(register_offset_TLR0, period)
mmio_0.write(register_offset_TLR1, pulse)

In [46]:
mask = 0xffffffff
gpio_out.write(0x0,mask)
mmio_0.write(register_offset_TLR1, 0)